# 한국어 뉴스 기사분류

- 자료실에 있는 BalancedNewsCorpus_train.csv, BalancedNewsCorpus_test.csv는 국어원 뉴스 자료에서 9개 분야의 신문별 균형을 맞춘 자료로, 학습용 9,000개 시험용 1800 자료가 있는 파일이다.
- 이 파일을 가지고 https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라 한국어 뉴스기사 분류기를 만들어라
- training data에서 evaluation data를 나누어 사용할 수 있다.(필요시)
- 화일 이름은 MidTermProject_DS(or CL)_Group X
- 조원 이름 명시

## 목표

- csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
(Field, Iterator, train,test, evaluation and prediction)
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기 

- 외부에서 학습된 한국어 단어 임베딩을 torchtext에 통합하여 사용하기 (word2vec, glove, fasttext 중 골라서 사용)
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 기사들의 분류 결과를 보이기

- 참고 사이트
    - https://pytorch.org/text/
    - http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    - https://github.com/pytorch/text
    - https://mc.ai/using-fine-tuned-gensim-word2vec-embeddings-with-torchtext-and-pytorch/

## 이 자료를 위해 사전학습된 임베딩
- 필요에 따라 선택하여 사용할 수 있음

### Word2vec 모델
 - 노트 : https://drive.google.com/file/d/1KOv901TPv5gepEdd4cCsJWoCHVaAV-A-/view?usp=sharing
 - Word2Vec 형태소 모델 : https://drive.google.com/file/d/1DDx6lRSTVULRFP3kslQLZsuoGZJOtoR1/view?usp=sharing
 - Word2Vec 어절 모델 : https://drive.google.com/file/d/1-RuEk-MhULduAbizgt3wjsMOCL3sM_pl/view?usp=sharing

- from gensim.models.keyedvectors import KeyedVectors
- Word2Vec_300D_space_model = KeyedVectors.load_word2vec_format(path + 'Word2Vec_300D_space.model', binary=False, encoding='utf-8')


### Faxttext 모델

- fasttext 형태소 모델: https://drive.google.com/file/d/1-EBaAtFK7chB6qqckKmLdghR62SebEYK/view?usp=sharing
- fasttext 어절 모델: https://drive.google.com/file/d/1-0D7Fe5oG_z9pQqOjkewtsuV_uVkh_dF/view?usp=sharing
- fasttext 형태소 자모 모델: https://drive.google.com/file/d/1-WW_qWQZ2q3Jj9fXXex82dYHWIMHGVri/view?usp=sharing
- fasttext 어절 자모 모델: https://drive.google.com/file/d/1-P2b8Dp09fZYO2Y__wjPNmS77PF7kfqV/view?usp=sharing


- from gensim.models.keyedvectors import KeyedVectors
- fasttext_model2 = KeyedVectors.load_word2vec_format(path + 'fasttext_morph_300.model', binary=False, encoding='utf-8')

## Glove 모델

- https://drive.google.com/drive/folders/1pzVO0jwx1Zf8p4hjf4JQn81XWzktsIdg?usp=sharing 


- from gensim.models.keyedvectors import KeyedVectors
- Glove_model = KeyedVectors.load_word2vec_format(# 모델 경로 , binary=False, encoding='utf-8')


## 정리
- 구현한 시스템의 성능을 정리


## Parameters

In [1]:
data_path = 'data/'
embedding_path = 'word_embeddings/'
batch_size= 128
gpu = 'cuda:0'

## Create Preprocessed data
#### Preprocessing from https://drive.google.com/drive/folders/1pzVO0jwx1Zf8p4hjf4JQn81XWzktsIdg?usp=sharing


In [2]:
import pandas as pd
import os

train_df = pd.read_csv(os.path.join(data_path + 'BalancedNewsCorpus_train.csv'), encoding='utf-8')
test_df = pd.read_csv(os.path.join(data_path + 'BalancedNewsCorpus_test.csv'), encoding='utf-8')

In [3]:
import re
import hanja

def cleaning_strings(input_text):

    input_text = input_text.replace('<p>',' ').replace('</p>','\n')  # 문단 간 구분이 필요 없으므로, 문단 구분자 삭제, 줄바꿈 삽입 
    input_text = input_text.translate(str.maketrans('①②③④⑤⑥⑦⑴⑵⑶⑷⑸ⅠⅡⅢ','123456712345123'))  # 숫자 정리
    input_text = input_text.translate(str.maketrans('―“”‘’〉∼\u3000', '-""\'\'>~ '))  # 유니코드 기호 정리
    input_text = input_text.translate({ord(i): None for i in '↑→↓⇒∇■□▲△▶▷▼◆◇○◎●★☆☞♥♪【】'})  # 특수문자 정리

    # 이메일 패턴 제거
    EMAIL_PATTERN = re.compile(r'(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))')
    input_text = re.sub(EMAIL_PATTERN, ' ', input_text)

    # url 패턴 제거
    URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
    input_text = re.sub(URL_PATTERN, ' ', input_text)
    
    input_text = re.sub('\(.+?\)','',input_text)  # 괄호 안 내용 삭제
    input_text = hanja.translate(input_text, 'substitution')  # 한자 -> 한글 치환
    input_text = re.sub('([\'\",.\(\)\[\]\{\}<\>\:\;\/\?\!\~\…\·\=\+\-\_])',' \g<1> ',input_text)  # 각종 문장부호 전후 띄어쓰기


    while True:
        temp_text = re.sub('(.+)([.|,])([가-힣]+)','\g<1>\g<2> \g<3>', input_text)  # 앞텍스트.뒷텍스트  처럼 마침표/쉼표 뒤에 띄어쓰기가 없는 경우 띄어쓰기
        if input_text == temp_text:                         # -> 재귀적으로 구현하여 여러번 시행
            input_text = temp_text
            break
        else:
            input_text = temp_text[:]

    input_text = re.sub('[0-9]+','NUM',input_text)  # 모든 숫자 NUM 으로 마스킹      
    input_text = re.sub('[ ]{2,}',' ', input_text)  # 띄어쓰기 2번 이상 중복된 경우 하나로 통합

    output_text = input_text.strip()

    return output_text

In [4]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''', re.VERBOSE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.VERBOSE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [5]:
def cleansing_other(sentence: str = None) -> str:
    """
    문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    
    return sentence

In [6]:
def cleansing_chinese(sentence: str = None) -> str:
    """
    한자를 변환하는 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    # chinese character를 앞뒤로 괄호가 감싸고 있을 경우, 대부분 한글 번역임
    sentence = re.sub("\([\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+\)", "", sentence)
    # 다른 한자가 있다면 한글로 치환
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sentence) is not None:
        sentence = hanja.translate(sentence, 'substitution')

    return sentence

In [7]:
def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

In [8]:
def cleansing_numbers(sentence: str = None) -> str:
    """
    숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [9]:
def preprocess_sent(sentence: str = None) -> str:
    """
    모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = sentence.replace('<p>',' ').replace('</p>',' ')
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_chinese(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)

    return sent_clean

In [10]:
# apply preprocessing
# train_df['News'] = train_df['News'].apply(cleaning_strings)
# test_df['News'] = test_df['News'].apply(cleaning_strings)
train_df['News'] = train_df['News'].apply(preprocess_sent)
test_df['News'] = test_df['News'].apply(preprocess_sent)

In [11]:
train_df

,filename,date,NewsPaper,Topic,News
0,NLRW1900000141,20170324,부산일보,스포츠,야구 종가 마침내 정상에 서다 야구 종가 미국이 푸에르토리코를 누르고 NUM월드베이...
1,NPRW1900000003,20110209,한국경제신문사,정치,외통위 NUM명중 NUM명 FTA 추가협상안만 처리 국회 외교통상통일위원회 소속 의...
2,NLRW1900000144,20100406,영남일보,사회,한나라 지선후보 희망연대 당원 구함 공천변수 작용 주목 오늘까지 추가 모집 오는 N...
3,NLRW1900000064,20100804,광주매일신문,스포츠,모처럼 살아난 CK포 NUM타점 합작 KIA NUMLG NUM강 진입을 놓고 혈전을...
4,NLRW1900000070,20160615,광주매일신문,문화,아문화전당서 동방의 등불 만나다 일찍이 아시아의 황금 시기에 빛나던 등불의 하나였던...
...,...,...,...,...,...
8995,NWRW1900000006,20141114,조선일보사,IT/과학,내가 구매한 영화 출근길의 오빠도 방에 있는 엄마도 보네 스마트폰 가족 공유 콘텐츠...
8996,NIRW1900000022,20101217,노컷뉴스,연예,꽃보다 예쁜 터치 선웅의 여장 이게 바로 안산 FNUM수퍼 루키 터치 TONUMCH...
8997,NLRW1900000092,20180131,국제신문,사회,어머니 빨리 쾌차하세요 밀양참사 후 더 깊어진 고부애 같은 병실 입원해 극진히 수발...
8998,NLRW1900000103,20090206,대전일보,IT/과학,엑스포공원 HD 드라마 타운 성공하려면 대전 엑스포 과학공원 내 조성될 것으로 기대...


## Load torch text

#### define custom dataset class


In [12]:
from torchtext.data import Field, Dataset, Example

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
            examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
                string is a field name, and the Field is the associated field.
            filter_pred (callable or None): use only examples for which
                filter_pred(example) is true, or use all examples if None.
                Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

    @staticmethod
    def sort_key(ex):
        return len(ex.News)

class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""
  
    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()
        
        for key, field in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex


In [13]:
# Use mecab as tokenizer
# Since all the pretrained embeddings used mecab (not sure mecab performs the best)
from konlpy.tag import Mecab
tokenizer = Mecab()

TEXT = Field(use_vocab=True, tokenize=tokenizer.morphs, include_lengths=True)
LABEL = Field(sequential=False, use_vocab=True, is_target=True, unk_token=None)
fields = { 'Topic' : LABEL, 'News' : TEXT }

In [14]:
train_dataset = DataFrameDataset(train_df, fields)
test_dataset = DataFrameDataset(test_df, fields)
TEXT.build_vocab(train_dataset, min_freq=10) 
LABEL.build_vocab(train_dataset)


In [15]:
from torchtext import data

train_loader = data.BucketIterator(
    dataset=train_dataset, batch_size=batch_size, device=gpu, sort_within_batch=True,
    train=True, repeat=False)
test_loader = data.BucketIterator(
    dataset=test_dataset, batch_size=batch_size, device=gpu,
    train=False, repeat=False)

In [16]:
train_loader.sort_key

<function __main__.DataFrameDataset.sort_key(ex)>

## Define Word Embedding
#### Word2Vec 300D token for now


In [17]:
from gensim.models.keyedvectors import KeyedVectors
# Word2Vec_300D_space_model = KeyedVectors.load_word2vec_format(embedding_path + 'Word2Vec_300D_space.model', binary=False, encoding='utf-8')
word_embeddings = KeyedVectors.load_word2vec_format(embedding_path  + 'Word2Vec_300D_token.model', binary=False, encoding='utf-8')

In [18]:
len(word_embeddings.index2word)

19716

In [19]:
len(TEXT.vocab)

19704

In [20]:
import torch
from torch import nn, optim

# add unk and pad
embeddings = nn.Embedding(num_embeddings=len(TEXT.vocab), embedding_dim=word_embeddings.vector_size)
nn.init.uniform_(embeddings.weight.data)
for i, w in enumerate(TEXT.vocab.itos):
    if w in word_embeddings:
        embeddings.weight.data[i] = torch.FloatTensor(word_embeddings[w])



In [21]:
from torch import nn
from torch.nn import init
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class BiRNNMax(nn.Module):

    def __init__(self, rnn_type, input_dim, hidden_dim, dropout_prob=0):
        super().__init__()
        self.rnn_type = rnn_type
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.dropout_prob = dropout_prob

        if rnn_type == 'gru':
            self.rnn = nn.GRU(
                input_size=input_dim, hidden_size=hidden_dim,
                bidirectional=True, dropout=dropout_prob)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(
                input_size=input_dim, hidden_size=hidden_dim,
                bidirectional=True, dropout=dropout_prob)
        else:
            raise ValueError('Unknown RNN type!')
        self.reset_parameters()

    def reset_parameters(self):
        init.orthogonal_(self.rnn.weight_hh_l0.data)
        init.kaiming_normal_(self.rnn.weight_ih_l0.data)
        init.constant_(self.rnn.bias_hh_l0.data, val=0)
        init.constant_(self.rnn.bias_ih_l0.data, val=0)
        init.orthogonal_(self.rnn.weight_hh_l0_reverse.data)
        init.kaiming_normal_(self.rnn.weight_ih_l0_reverse.data)
        init.constant_(self.rnn.bias_hh_l0_reverse.data, val=0)
        init.constant_(self.rnn.bias_ih_l0_reverse.data, val=0)
        if self.rnn_type == 'lstm':
            # Set the initial forget bias values to 1
            self.rnn.bias_ih_l0.data.chunk(4)[1].fill_(1)
            self.rnn.bias_ih_l0_reverse.data.chunk(4)[1].fill_(1)

    def forward(self, inputs, length):
        """
        Args:
            inputs (Variable): A float variable of size
                (max_length, batch_size, input_dim).
            length (Tensor): A long tensor of sequence lengths.

        Returns:
            output (Variable): An encoded sequence vector of size
                (batch_size, hidden_dim).
        """

        inputs_packed = pack_padded_sequence(inputs, lengths=list(length))
        rnn_outputs_packed, _ = self.rnn(inputs_packed)
        rnn_outputs, _ = pad_packed_sequence(rnn_outputs_packed)
        # To avoid the weired bug when taking the max of a length-1 sentence.
        output = rnn_outputs.max(dim=0, keepdim=True)[0].squeeze(0)
        return output


In [22]:
class BiRNNTextClassifier(nn.Module):

    def __init__(self, rnn_type, num_classes, word_dim, hidden_dim,
                 clf_dim, dropout_prob=0):
        super().__init__()
        self.rnn_type = rnn_type
        self.num_classes = num_classes
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.clf_dim = clf_dim
        self.dropout_prob = dropout_prob

        self.dropout = nn.Dropout(dropout_prob)

        self.birnn_max = BiRNNMax(
            rnn_type=rnn_type, input_dim=word_dim, hidden_dim=hidden_dim,
            dropout_prob=dropout_prob)
        self.clf = nn.Sequential(
            nn.Linear(in_features=2 * hidden_dim, out_features=clf_dim),
            nn.ReLU(),
            self.dropout,
            nn.Linear(in_features=clf_dim, out_features=num_classes))
        self.reset_parameters()

    def reset_parameters(self):
        self.birnn_max.reset_parameters()
        init.kaiming_normal_(self.clf[0].weight.data)
        init.constant_(self.clf[0].bias.data, val=0)
        init.uniform_(self.clf[3].weight.data, -0.005, 0.005)
        init.constant_(self.clf[3].bias.data, val=0)

    def forward(self, inputs, length, batch_first=False):
        """
        Args:
            inputs (Variable):
                If use_pretrained_embeddings is False, this is a long
                    variable of size (max_length, batch_size) or
                    (batch_size, max_length) (if batch_first) which
                    contains indices of words.
                If use_pretrained_embeddings is True, this is a 3D
                    variable of size (max_length, batch_size, word_dim)
                    or (batch_size, max_length, word_dim).
            length (Tensor): A long tensor of lengths.
            batch_first (bool): If True, sequences in a batch are
                aligned along the first dimension of inputs.

        Returns:
            logit (Variable): A variable containing unnormalized log
                probability for each class.
        """

        if batch_first:
            inputs = inputs.transpose(0, 1)
        inputs = self.dropout(inputs)
        sentence_vector = self.birnn_max(inputs=inputs, length=length)
        sentence_vector = self.dropout(sentence_vector)
        logit = self.clf(sentence_vector)
        return logit

In [23]:
from torch import optim

model = BiRNNTextClassifier(
        rnn_type='lstm', num_classes=len(LABEL.vocab), word_dim=300,
        hidden_dim=300, clf_dim=300,
        dropout_prob=0.0)
optimizer = optim.Adam(list(model.parameters()))
loss_weight = None
criterion = nn.CrossEntropyLoss(weight=loss_weight)
embeddings.cuda(gpu)
model.cuda(gpu)
criterion.cuda(gpu)

CrossEntropyLoss()

In [26]:
from torch.nn.utils import clip_grad_norm_

def run_iter(batch):
    inputs, length = batch.News
    inputs = embeddings(inputs)
    logit = model(inputs=inputs, length=list(length))

    label = batch.Topic
    loss = criterion(input=logit, target=label)
    accuracy = torch.eq(logit.max(1)[1], label).float().mean()
    if model.training:
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()
    return loss.data.item(), accuracy.data.item()

def validate():
    loss_sum = accuracy_sum = 0
    num_batches = len(test_loader)
    model.eval()
    for valid_batch in test_loader:
        loss, accuracy = run_iter(valid_batch)
        loss_sum += loss
        accuracy_sum += accuracy
    return loss_sum / num_batches, accuracy_sum / num_batches

iter_count = 0
best_valid_accuracy = -1
for cur_epoch in range(50):
    for train_batch in train_loader:
        if not model.training:
            model.train()
        train_loss, train_accuracy = run_iter(train_batch)
        iter_count += 1

    print(f'* Epoch {cur_epoch} finished')
    valid_loss, valid_accuracy = validate()
    print(f'  - Valid Loss = {valid_loss:.4f}')
    print(f'  - Valid Accuracy = {valid_accuracy:.4f}')
    # if valid_accuracy > best_valid_accuracy:
    #     best_valid_accuracy = valid_accuracy
    #     model_filename = (f'model-{cur_epoch}-{valid_accuracy:.4f}.pt')
    #     model_path = os.path.join(args.save_dir, model_filename)
    #     state_dict = 'model':  model.state_dict()
    #     torch.save(state_dict, model_path)
    #     torch.save(state_dict, os.path.join(args.save_dir,'final.pt'))
    #     print(f'  - Saved the new best model to {model_path}')

* Epoch 0 finished
  - Valid Loss = 1.3191
  - Valid Accuracy = 0.4995
* Epoch 1 finished
  - Valid Loss = 1.2649
  - Valid Accuracy = 0.4995
* Epoch 2 finished
  - Valid Loss = 1.2061
  - Valid Accuracy = 0.5271
* Epoch 3 finished
  - Valid Loss = 1.2115
  - Valid Accuracy = 0.5495
* Epoch 4 finished
  - Valid Loss = 1.1540
  - Valid Accuracy = 0.5641
* Epoch 5 finished
  - Valid Loss = 1.2072
  - Valid Accuracy = 0.5000
* Epoch 6 finished
  - Valid Loss = 1.0308
  - Valid Accuracy = 0.5625
* Epoch 7 finished
  - Valid Loss = 1.0510
  - Valid Accuracy = 0.6089
* Epoch 8 finished
  - Valid Loss = 0.9245
  - Valid Accuracy = 0.6792
* Epoch 9 finished
  - Valid Loss = 0.9672
  - Valid Accuracy = 0.6495
* Epoch 10 finished
  - Valid Loss = 0.8658
  - Valid Accuracy = 0.7016
* Epoch 11 finished
  - Valid Loss = 0.7278
  - Valid Accuracy = 0.7635
* Epoch 12 finished
  - Valid Loss = 0.7680
  - Valid Accuracy = 0.7250
* Epoch 13 finished
  - Valid Loss = 0.6861
  - Valid Accuracy = 0.7656
* 

## User Input

####  뉴스 labels
    -  IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8

In [25]:
def predict_news(model, sentence, min_len=5):

SyntaxError: unexpected EOF while parsing (<ipython-input-25-45277e230217>, line 1)

In [40]:
## 아래 문장의 정답은 5-8-1-2-4-6-7-3-0
## 연예/문화, 정치/경제/사회/생활 등 명확히 구별되기 어려운 범주들이 있음...

sentence = "차진 식감과 부드러운 감촉을 모두 지닌 식빵, 결결이 찢어지는 크루아상, 둥글고 크게 구운 호밀빵 모두 모양새부터 알차고 단단했다. 디저트로 눈을 옮기면 국가 대표팀같이 뭐 하나 빼놓을 수 없는 케이크가 나란히 줄을 서 있었다."
sentence ="정부는 2012년 예산의 공고안과 배정계획을 1월3일 국무회의에서 의결하고 연초부터 바로 집행에 들어간다. 세계 경제의 불확실성이 높아지고 경기가 둔화할 가능성이 높은 만큼 조기 집행에 박차를 가할 예정이다"
sentence = "국세청은 특히 서민생활에 피해를 주면서 폭리를 취하는 매점매석 농수산물 유통업체 등에 대한 추적조사를 강화하고, 지방청에 ‘민생침해 사업자 조사전담팀’을 꾸려 민생침해 탈세자에 대한 엄정 대응에 나설 계획이라고 밝혔다"
sentence = "26일 제25회 부산국제영화제 갈라 프레젠테이션 부문 초청작 '스파이의 아내' 온라인 기자회견이 진행됐다. 작품을 연출한 구로사와 감독이 참석해 작품에 대한 이야기를 나눴다."
sentence = "70대 운전사가 몰던 25인승 어린이 통학버스가 주유소로 돌진해 차량 3대를 들이 받았다. 다행히 통학버스에 운전자 외에는 탑승자가 없어 큰 인명피해는 피했다. 운전자는 차량 결함을 주장하고 있으나, 경찰은 운전자 과실 여부도 조사 중이다."
sentence = "토트넘이 손흥민에게 주급 20만 파운드(약 3억원)-5년 재계약을 제안할 준비를 마쳤다.' 25일(한국시각) 영국 대중일간 더선의 헤드라인이다. 조제 무리뉴 토트넘 감독이 번리전을 앞두고 기자회견을 통해 구단에 토트넘에서의 손흥민의 장밋빛 미래를 확신하며 재계약을 요청한 직후 영국 현지 언론에선 손흥민 재계약 보도가 쏟아지고 있다."
sentence = "공연은 말 그대로 다채로움 그 자체였다. 발레극인지, 현대무용극인지, 전통극인지, 연극인지, 연극이면 다인극인지 1인극인지 모를 정도로 다양한 장르의 결합이 먼저 눈에 띈다."
sentence = "발이 아프면 걷는 자세가 나빠지고 자연스럽게 무릎, 골반, 허리에 이상이 생길 수 있다. 이를 예방하려면 평소 발바닥 근육을 스트레칭하고 강화하는 운동을 지속하는 게 중요하다."
sentence = "전기차 화재 사고가 연이어 발생하는 가운데 불타지 않는 SK이노베이션 배터리가 주목받는다. SK이노베이션의 배터리는 글로벌 배터리 업체 중 유일하게 단 한건의 화재도 일어나지 않았다. 이같은 비결이 자동차 안정성을 결정짓는 분리막 내재화에 있다는 평가가 나온다."
predict_news(model, sentence)

NameError: name 'predict_news' is not defined